In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split as Tts
import string
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2023-01-15 13:36:58--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0s      

2023-01-15 13:36:58 (94.8 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2023-01-15 13:36:58--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP re

Extract Poem data and add to list

In [ ]:
files = ['/content/edgar_allan_poe.txt', '/content/robert_frost.txt' ]

lis_line = []
lis_author = []

for author, file in enumerate(files):
  for line in open(file):
    line = line.lower().rstrip()
    if line:
      line = line.translate(str.maketrans('', '', string.punctuation))
      lis_line.append(line)
      lis_author.append(author)

len(lis_line)

2154

Split into Training and Test Data, Add unique words to dictionary

In [ ]:
#Split into training and test set
lis_line_train, lis_line_test, lis_author_train, lis_author_test = Tts(lis_line, lis_author, random_state=1)

#Add all unique words to a dictionary map
dict_uni_words = {'<unk>': 0}; word_count = 0

for line in lis_line_train:
  for word in line.split():
    if word not in dict_uni_words.keys():
      word_count += 1
      dict_uni_words[word] = word_count

Map words to integers

In [ ]:
#map words to integers for training data
lis_line_train_int = []

for line in lis_line_train:
  conv = [dict_uni_words[word] for word in line.split()]
  lis_line_train_int.append(conv)

#map words to integers for test data
lis_line_test_int = []

for line in lis_line_test:
  conv = [dict_uni_words.get(word, 0) for word in line.split()]
  lis_line_test_int.append(conv)

In [ ]:
lis_line_train_int[100:105]

[[6, 388, 20, 389],
 [11, 390, 391, 392, 77, 34, 36, 6],
 [53, 26, 393],
 [323, 269, 236, 394, 395, 18, 6, 396],
 [34, 397, 323, 398, 42, 399]]

Estimate the initial state distribution, $\hat{\pi}$

In [ ]:
lis_pi_poe = [] ; lis_pi_frost = []

for word_index in dict_uni_words.values():
  count_poe = 0; count_frost = 0
  
  for line, author in zip(lis_line_train_int, lis_author_train):
    if author == 0 and line[0] == word_index: #poe
      count_poe += 1
    elif author == 1 and line[0] == word_index: #frost
      count_frost +=1

  lis_pi_poe.append((1 + count_poe)/(lis_author_train.count(0) + len(dict_uni_words)))
  lis_pi_frost.append((1 + count_frost)/(lis_author_train.count(1) + len(dict_uni_words)))

arr_pi_poe = np.array(lis_pi_poe)
arr_pi_frost = np.array(lis_pi_frost)

Estimate the state transition Matrix, $\hat{A}_{ij}$



In [ ]:
arr_A_poe = np.ones((len(dict_uni_words), len(dict_uni_words) ))
arr_A_frost = np.ones((len(dict_uni_words), len(dict_uni_words) ))

for line, author in zip(lis_line_train_int, lis_author_train):
  for prev_word, word in zip(line[0:-1],line[1:]):
    if author == 0: #poe
      arr_A_poe[prev_word, word] += 1
    elif author == 1: #frost
      arr_A_frost[prev_word, word] += 1     

arr_A_poe /= arr_A_poe.sum(axis=1, keepdims=True)
arr_A_frost /= arr_A_frost.sum(axis=1, keepdims=True)

Calculate the Prior

In [ ]:
p_poe = lis_author_train.count(0)/len(lis_author_train)
p_frost = lis_author_train.count(1)/len(lis_author_train)

p_poe, p_frost

(0.33126934984520123, 0.6687306501547987)

Calculate Log Probabilities

In [ ]:
arr_pi_poe_log = np.log(arr_pi_poe)
arr_A_poe_log = np.log(arr_A_poe)

arr_pi_frost_log = np.log(arr_pi_frost)
arr_A_frost_log = np.log(arr_A_frost)

p_poe_log = np.log(p_poe)
p_frost_log = np.log(p_frost)

In [ ]:
class Fish:
    def __init__(self,var1):
      self.fish_var = var1
    
    def print_stuff(self):
      print(self.fish_var)



In [2]:
a  = [0.02, {'a':0.5,'b':0.6}]

Write classifer

In [ ]:
class Classifier:

  def setup(self, logp0, logp1, logA0, logA1, logauth0, logauth1, dict_map):
    self.logp0 = logp0
    self.logp1 = logp1
    self.logA0 = logA0
    self.logA1 = logA1
    self.logauth0 = logauth0
    self.logauth1 = logauth1
    self.dict_map = dict_map
  
  def predict(self, lis_str):  #accepts a list of strings
    #encode list of strings to integer format
    lis_str_int = []

    for line in lis_str:
      lis_line_int = []
      for word in line.split():
        lis_line_int.append(self.dict_map.get(word, 0))
      
      lis_str_int.append(lis_line_int)
    
    #Loop through each integer encoded string
    auth = []

    for line in lis_str_int:

      #for each look up estimate initial state log probability from lists, 0 poe, 1 frost
      logp0_poe = self.logp0[line[0]]
      logp1_frost = self.logp1[line[0]]

      #for each author lookup string log probability
      logA0_poe = []; logA1_frost = []
      
      for i, j in zip(line[0:-1],line[1:]):
        logA0_poe.append(self.logA0[i,j])
        logA1_frost.append(self.logA1[i,j])

      #Sum log probabilities for transition matrix
      logA0_poe = sum(logA0_poe)
      logA1_frost = sum(logA1_frost)

      #Sum log probabilities
      logp_poe = logA0_poe + logp0_poe + self.logauth0
      logp_frost = logA1_frost + logp1_frost + self.logauth1

      if logp_poe > logp_frost:
        auth.append(0)
      elif logp_poe < logp_frost:
        auth.append(1)

    return(auth)
       


In [ ]:
test = Classifier()
test.setup(arr_pi_poe_log, arr_pi_frost_log, arr_A_poe_log, arr_A_frost_log, p_poe_log,p_frost_log, dict_uni_words)
out = test.predict(lis_line_test)

In [ ]:
np.mean(np.array(lis_author_test) == np.array(out))

0.8163265306122449

In [ ]:
cm = confusion_matrix(lis_author_test, out)
cm

array([[ 95,  88],
       [ 11, 345]])

# Unused Code

In [ ]:
#Faster loop
%%time

lis_uni_words = []

for line in list(df_x_train):
  lis_uni_words += line.split()

lis_uni_words = list(set(lis_uni_words))

dict_uni_words_2 = {}
word_count = 0

for word in lis_uni_words:
   word_count += 1
   dict_uni_words[word_count] = word